In [1]:
library(tidyverse)
library(lubridate)

library(assertr)

library(ggtext)


── Attaching packages ────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ───────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Notes


#### Legal (ISO) gender types:

* https://data.gov.uk/education-standards/sites/default/files/CL-Legal-Sex-Type-v2-0.pdf


#### For data stored as % 

* need to relax sum to 100%
* 			
Symbol	Meaning		
    * '-'	Not Applicable		
    * '-'	No Entries (Table 3)		
    * 0%	Less than 0.5%		
    * ***	Fewer Than 5 Entries	

<br>&nbsp;
<h3>Error Checking &amp; Warnings</h3>

* Ideally correct errors here and write out corrected csv to file with a note
* TODO - log errors found and include error-checking code as part of pre-processing flow



<h3>Errors to Watch For</h3>

<b>Please document as not found and/or what corrected, so can trace back to original. 
Update as needed and mirror in final docs submitted with project.</b>

* "Computing" (or "Computing Studies" or "Computing (New)") ... included in list of subjects
    * need to decide if files will be excluded or included with a flag to track changes in subjects offered
* Each subject and grade listed only once per gender
* proportions of male/female add up to 1
<br />


<h3>Warning Only Needed</h3>

<b>Need only document if triggered.</b>

* All values for a subject set to "-" or 0 (rare) -> translates to NAs if read in properly

<br />

In [2]:
files_to_verify <- list.files("sta_it_402/data", pattern = "higher", full.names = T, 
                              ignore.case = T, recursive = T) #%>%
                        #as.data.frame
files_to_verify

[1] "sta_it_402/data/grades/1986_Higher.csv"         
 [2] "sta_it_402/data/grades/1987_Higher.csv"         
 [3] "sta_it_402/data/grades/1988_Higher.csv"         
 [4] "sta_it_402/data/grades/1989_Higher.csv"         
 [5] "sta_it_402/data/grades/1990_Higher.csv"         
 [6] "sta_it_402/data/grades/1991_Higher.csv"         
 [7] "sta_it_402/data/grades/1992_Higher.csv"         
 [8] "sta_it_402/data/grades/1993_Higher.csv"         
 [9] "sta_it_402/data/grades/1994_Higher.csv"         
[10] "sta_it_402/data/grades/1995_Higher.csv"         
[11] "sta_it_402/data/grades/1999_Higher.csv"         
[12] "sta_it_402/data/grades/2000_NewHigher.csv"      
[13] "sta_it_402/data/grades/2005_AdvancedHigher.csv" 
[14] "sta_it_402/data/grades/2005_Higher_National.csv"
[15] "sta_it_402/data/grades/2012_AdvancedHigher.csv" 
[16] "sta_it_402/data/grades/2012_Higher.csv"         
[17] "sta_it_402/data/grades/2019_AdvancedHigher.csv" 
[18] "sta_it_402/data/grades/2019_Higher.csv"

In [3]:
focus_subject <- "computing"
redundant_column_flags <- c("-Passes", "-percentage*", "-COMP", "-PassesUngradedCourses")

In [4]:
# check focus subject (typically, but not necessarily, Computing) in list of subjects

checkFocusSubjectListed <- 
    function(awardFile, glimpseContent = FALSE, listSubjects = FALSE) {
        awardData <- read_csv(awardFile, trim_ws = T) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                            filter(rowSums(is.na(.)) != ncol(.)) %>%
                            suppressMessages
         
        print(awardFile)
        if (!exists("focus_subject") || is_null(focus_subject) || (str_trim(focus_subject) == "")) {
            focus_subject <- "computing"
            print(paste("No focus subject specified; defaulting to subjects containing: ", focus_subject))
            
        } else 
            print(paste("Search on focus subject (containing term) '", focus_subject, "'", sep = ""))
        
        if (glimpseContent)
            print(glimpse(awardData))
        
        result <- awardData %>%
            select(Subject) %>%

            filter(str_detect(Subject, regex(focus_subject, ignore_case = TRUE))) %>%
            verify(nrow(.) > 0, error_fun = just_warn) 
        
        if (!listSubjects)    
            return(nrow(result)) # comment out this row to list subject names
        else
            return(result)
    }

In [5]:
#lapply(files_to_verify, checkFocusSubjectListed, listSubjects = TRUE)
#Map(checkFocusSubjectListed, files_to_verify, listSubjects = TRUE)

as.data.frame(sapply(files_to_verify, checkFocusSubjectListed)) # call without as.data.frame if listing values

[1] "sta_it_402/data/grades/1986_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
verification [nrow(.) > 0] failed! (1 failure)

    verb redux_fn   predicate column index value
1 verify       NA nrow(.) > 0     NA     1    NA



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1987_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
verification [nrow(.) > 0] failed! (1 failure)

    verb redux_fn   predicate column index value
1 verify       NA nrow(.) > 0     NA     1    NA



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1988_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
verification [nrow(.) > 0] failed! (1 failure)

    verb redux_fn   predicate column index value
1 verify       NA nrow(.) > 0     NA     1    NA



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1989_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1990_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1991_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1992_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1993_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1994_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1995_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/1999_Higher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/2000_NewHigher.csv"
[1] "Search on focus subject (containing term) 'computing'"
[1] "sta_it_402/data/grades/2005_AdvancedHigher.csv"

,"sapply(files_to_verify, checkFocusSubjectListed)"
,<int>
sta_it_402/data/grades/1986_Higher.csv,0
sta_it_402/data/grades/1987_Higher.csv,0
sta_it_402/data/grades/1988_Higher.csv,0
sta_it_402/data/grades/1989_Higher.csv,1
sta_it_402/data/grades/1990_Higher.csv,1
sta_it_402/data/grades/1991_Higher.csv,1
sta_it_402/data/grades/1992_Higher.csv,1
sta_it_402/data/grades/1993_Higher.csv,1
sta_it_402/data/grades/1994_Higher.csv,1


In [6]:
# check for data stored as percentages only

checkDataAsPercentageOnly <- 
    function(awardFile, glimpseContent = F) {
        awardData <- read_csv(awardFile, trim_ws = T) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                            filter(rowSums(is.na(.)) != ncol(.)) %>%
                            suppressMessages
        
        print(awardFile)
        if (glimpseContent)
            print(glimpse(awardData))
        
        if (!exists("redundant_column_flags") || is.null(redundant_column_flags)) 
            redundant_column_flags <- c("-percentage*", "-COMP", "-PassesUngradedCourses")
        
        awardData %>%
            select(-matches(redundant_column_flags)) %>% # "-percentage")) %>%
            select(matches(c("male-", "female-"))) %>%
            verify(ncol(.) > 0, error_fun = just_warn) %>%

            summarise(data_as_counts = (ncol(.) > 0))
    }

In [7]:
sapply(files_to_verify, checkDataAsPercentageOnly)
#Map(checkDataAsPercentageOnly, files_to_verify) #, T)


[1] "sta_it_402/data/grades/1986_Higher.csv"
[1] "sta_it_402/data/grades/1987_Higher.csv"
[1] "sta_it_402/data/grades/1988_Higher.csv"
[1] "sta_it_402/data/grades/1989_Higher.csv"
[1] "sta_it_402/data/grades/1990_Higher.csv"
[1] "sta_it_402/data/grades/1991_Higher.csv"
[1] "sta_it_402/data/grades/1992_Higher.csv"
[1] "sta_it_402/data/grades/1993_Higher.csv"
[1] "sta_it_402/data/grades/1994_Higher.csv"
[1] "sta_it_402/data/grades/1995_Higher.csv"
[1] "sta_it_402/data/grades/1999_Higher.csv"
[1] "sta_it_402/data/grades/2000_NewHigher.csv"
[1] "sta_it_402/data/grades/2005_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2005_Higher_National.csv"
[1] "sta_it_402/data/grades/2012_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2012_Higher.csv"
[1] "sta_it_402/data/grades/2019_AdvancedHigher.csv"
verification [ncol(.) > 0] failed! (1 failure)

    verb redux_fn   predicate column index value
1 verify       NA ncol(.) > 0     NA     1    NA



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/2019_Higher.csv"
verification [ncol(.) > 0] failed! (1 failure)

    verb redux_fn   predicate column index value
1 verify       NA ncol(.) > 0     NA     1    NA



Warning message:
“assertr encountered errors”


$`sta_it_402/data/grades/1986_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1987_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1988_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1989_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1990_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1991_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1992_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1993_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1994_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1995_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/1999_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2000_NewHigher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2005_AdvancedHigher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2005_Higher_National.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2012_AdvancedHigher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2012_Higher.csv.data_as_counts`
[1] TRUE

$`sta_it_402/data/grades/2019_AdvancedHigher.csv.data_as_counts`
[1] FALSE

$`sta_it_402/data/grades/2019_Higher.csv.data_as_counts`
[1] FALSE

In [8]:
# error checking - need to manually correct data if mismatch between breakdown by gender and totals found
# this case, if found, is relatively easy to fix

checkDistributionByGenderErrors <- 
    function(awardFile, glimpseContent = FALSE) {
        awardData <- read_csv(awardFile, trim_ws = TRUE) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                            filter(rowSums(is.na(.)) != ncol(.)) %>%
                            suppressMessages
        
        print(awardFile)
        if (glimpseContent)
            print(glimpse(awardData))

        
        awardData <- awardData %>%
            mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("all-")), as.character) %>%
            mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("all-")), parse_number) %>%
            suppressWarnings


        data_as_counts <- awardData %>%
                    select(-matches(redundant_column_flags)) %>% # "-percentage")) %>%
                    select(matches(c("male-", "female-"))) %>%

                    summarise(data_as_counts = (ncol(.) > 0)) %>%
                    as.logical


        if (data_as_counts) {

            awardData <- awardData %>%

                select(-NumberOfCentres) %>%
                mutate_at(vars(starts_with("male")), ~(. / `all-Entries`)) %>%
                mutate_at(vars(starts_with("female")), ~(. / `all-Entries`)) %>%
                select(-(starts_with("all") & !ends_with("-Entries"))) %>%

                pivot_longer(!c(Subject), names_to = "grade", values_to = "PercentageOfStudents") %>%
                separate("grade", c("gender", "grade")) %>%
                mutate_at(c("gender", "grade"), as.factor) %>%
                filter(!(gender %in% c("all")) & (grade %in% c("Entries")))


        } else { # dataAsPercentageOnly

            awardData <- awardData %>%

                select(Subject, ends_with("-percentage")) %>%
                mutate_at(vars(ends_with("-percentage")), ~(. / 100)) %>%


                pivot_longer(!c(Subject), names_to = "grade", values_to = "PercentageOfStudents") %>%
                separate("grade", c("gender", "grade"), extra = "merge") %>%
                mutate_at(c("gender", "grade"), as.factor)

        } # end if-else - check for data capture approach
        

        awardData %>%

            group_by(Subject) %>%
            summarise(total = sum(PercentageOfStudents, na.rm = TRUE)) %>%
            verify((total == 1.0) | (total == 0), error_fun = just_warn) %>% 

            mutate(DataError = if_else(((total == 1.0) | (total == 0)), FALSE, TRUE)) %>%
            filter(DataError == TRUE) %>% # confirmation only - comment out to print all
            suppressMessages # ungrouping messages

}


In [9]:
# suppressWarnings(checkDistributionByGenderErrors("sta_it_402/data/grades/1989_Higher.csv")) #, TRUE))

data.frame(sapply(files_to_verify, checkDistributionByGenderErrors))


[1] "sta_it_402/data/grades/1986_Higher.csv"
[1] "sta_it_402/data/grades/1987_Higher.csv"
[1] "sta_it_402/data/grades/1988_Higher.csv"
[1] "sta_it_402/data/grades/1989_Higher.csv"
[1] "sta_it_402/data/grades/1990_Higher.csv"
[1] "sta_it_402/data/grades/1991_Higher.csv"
[1] "sta_it_402/data/grades/1992_Higher.csv"
[1] "sta_it_402/data/grades/1993_Higher.csv"
[1] "sta_it_402/data/grades/1994_Higher.csv"
[1] "sta_it_402/data/grades/1995_Higher.csv"
[1] "sta_it_402/data/grades/1999_Higher.csv"
[1] "sta_it_402/data/grades/2000_NewHigher.csv"
[1] "sta_it_402/data/grades/2005_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2005_Higher_National.csv"
[1] "sta_it_402/data/grades/2012_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2012_Higher.csv"
[1] "sta_it_402/data/grades/2019_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2019_Higher.csv"
verification [(total == 1) | (total == 0)] failed! (1 failure)

    verb redux_fn                   predicate column index value
1 verify       NA (total 

Warning message:
“assertr encountered errors”


,sta_it_402.data.grades.1986_Higher.csv,sta_it_402.data.grades.1987_Higher.csv,sta_it_402.data.grades.1988_Higher.csv,sta_it_402.data.grades.1989_Higher.csv,sta_it_402.data.grades.1990_Higher.csv,sta_it_402.data.grades.1991_Higher.csv,sta_it_402.data.grades.1992_Higher.csv,sta_it_402.data.grades.1993_Higher.csv,sta_it_402.data.grades.1994_Higher.csv,sta_it_402.data.grades.1995_Higher.csv,sta_it_402.data.grades.1999_Higher.csv,sta_it_402.data.grades.2000_NewHigher.csv,sta_it_402.data.grades.2005_AdvancedHigher.csv,sta_it_402.data.grades.2005_Higher_National.csv,sta_it_402.data.grades.2012_AdvancedHigher.csv,sta_it_402.data.grades.2012_Higher.csv,sta_it_402.data.grades.2019_AdvancedHigher.csv,sta_it_402.data.grades.2019_Higher.csv
,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
Subject,,,,,,,,,,,,,,,,,,Photography
total,,,,,,,,,,,,,,,,,,0.99
DataError,,,,,,,,,,,,,,,,,,TRUE


In [10]:

# warning only - document if necessary
# double-check for subjects with values all NA - does this mean subject being excluded or no one took it?

checkSubjectsWithNoEntries <- 
    function(awardFile, glimpseContent = F) {
        awardData <- read_csv(awardFile, trim_ws = T) %>% #, skip_empty_rows = T) # NOT skipping empty rows... :(
                            filter(rowSums(is.na(.)) != ncol(.)) %>%
                            suppressMessages
        
        print(awardFile)
        if (glimpseContent)
            print(glimpse(awardData))
        
        bind_cols(
            awardData %>%
                mutate(row_id = row_number()) %>%
                select(row_id, Subject), 
                  
            awardData %>%
                select(-c(Subject, NumberOfCentres)) %>%
                mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("all-")), as.character) %>%
                mutate_at(vars(starts_with("male-") | starts_with("female-") | starts_with("all-")), parse_number) %>%
                suppressWarnings %>%
 
                assert_rows(num_row_NAs, 
                    within_bounds(0, length(colnames(.)), include.upper = F), everything(), error_fun = just_warn) %>% 
                    # comment out just_warn to stop execution on fail
                summarise(column_count = length(colnames(.)),
                          count_no_entries = num_row_NAs(.)) 
                  
        ) %>% # end bind_cols
            
        filter(count_no_entries == column_count) # comment out to print all
    }

In [11]:
data.frame(sapply(files_to_verify, checkSubjectsWithNoEntries))

[1] "sta_it_402/data/grades/1986_Higher.csv"
[1] "sta_it_402/data/grades/1987_Higher.csv"
[1] "sta_it_402/data/grades/1988_Higher.csv"
Data frame row reduction 'num_row_NAs' violates predicate 'within_bounds(0, length(colnames(.)), include.upper = F)' 1 time
         verb    redux_fn
1 assert_rows num_row_NAs
                                                 predicate        column index
1 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()    20
  value
1    18



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1989_Higher.csv"
Data frame row reduction 'num_row_NAs' violates predicate 'within_bounds(0, length(colnames(.)), include.upper = F)' 1 time
         verb    redux_fn
1 assert_rows num_row_NAs
                                                 predicate        column index
1 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()    21
  value
1    18



Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1990_Higher.csv"
Data frame row reduction 'num_row_NAs' violates predicate 'within_bounds(0, length(colnames(.)), include.upper = F)' 1 time
         verb    redux_fn
1 assert_rows num_row_NAs
                                                 predicate        column index
1 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()    23
  value
1    18



...

Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1995_Higher.csv"
Data frame row reduction 'num_row_NAs' violates predicate 'within_bounds(0, length(colnames(.)), include.upper = F)' 20 times
         verb    redux_fn
1 assert_rows num_row_NAs
2 assert_rows num_row_NAs
3 assert_rows num_row_NAs
4 assert_rows num_row_NAs
5 assert_rows num_row_NAs
                                                 predicate        column index
1 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()     1
2 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()     7
3 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()     9
4 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()    12
5 within_bounds(0, length(colnames(.)), include.upper = F) ~everything()    14
  value
1    18
2    18
3    18
4    18
5    18
  [omitted 15 rows]




Warning message:
“assertr encountered errors”


[1] "sta_it_402/data/grades/1999_Higher.csv"
[1] "sta_it_402/data/grades/2000_NewHigher.csv"
[1] "sta_it_402/data/grades/2005_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2005_Higher_National.csv"
[1] "sta_it_402/data/grades/2012_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2012_Higher.csv"
[1] "sta_it_402/data/grades/2019_AdvancedHigher.csv"
[1] "sta_it_402/data/grades/2019_Higher.csv"


,sta_it_402.data.grades.1986_Higher.csv,sta_it_402.data.grades.1987_Higher.csv,sta_it_402.data.grades.1988_Higher.csv,sta_it_402.data.grades.1989_Higher.csv,sta_it_402.data.grades.1990_Higher.csv,sta_it_402.data.grades.1991_Higher.csv,sta_it_402.data.grades.1992_Higher.csv,sta_it_402.data.grades.1993_Higher.csv,sta_it_402.data.grades.1994_Higher.csv,sta_it_402.data.grades.1995_Higher.csv,sta_it_402.data.grades.1999_Higher.csv,sta_it_402.data.grades.2000_NewHigher.csv,sta_it_402.data.grades.2005_AdvancedHigher.csv,sta_it_402.data.grades.2005_Higher_National.csv,sta_it_402.data.grades.2012_AdvancedHigher.csv,sta_it_402.data.grades.2012_Higher.csv,sta_it_402.data.grades.2019_AdvancedHigher.csv,sta_it_402.data.grades.2019_Higher.csv
,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
row_id,,,20,21,23,46,57,"17, 73, 74, 75","6, 11, 17, 54, 58, 76, 77, 78, 79, 80","1, 7, 9, 12, 14, 17, 19, 30, 46, 49, 52, 54, 55, 66, 67, 68, 69, 70, 71, 72",,,,,,,,
Subject,,,Hebrew,Greek (Optional Paper III)*,Greek (Optional Paper III)*,Metalwork,Music Part II*,"Latin (Optional Paper III)*, Music Part II* , Music Part III* , Music Part IV*","Gáidhlig , Greek (Optional Paper III)* , Latin (Optional Paper III)* , Craft and Design , Home Economics (Fabrics and Fashion), Music Part II* , Music Part III* , Music Part IV* , Music Part V* , Music Part VI*","Classical Greek (Optional Paper III) , Gáidhlig , Gaelic (Learners) , Hebrew , Italian , Portuguese , Russian , Chemistry , Agricultural Science , Engineering , Horticultural Science , Navigation , Secretarial Studies (Audio-typewriting), Music (Practical Harmony)* , Music Part II* , Music Part III* , Music Part IV* , Music Part V* , Music Part VI* , Music",,,,,,,,
column_count,,,18,18,18,18,18,"18, 18, 18, 18","18, 18, 18, 18, 18, 18, 18, 18, 18, 18","18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18",,,,,,,,
count_no_entries,,,18,18,18,18,18,"18, 18, 18, 18","18, 18, 18, 18, 18, 18, 18, 18, 18, 18","18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18",,,,,,,,
